In [21]:
import os

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
root_dir = '../slices'

In [3]:
example_record = np.load("../slices/fold1/7061-6-0-0.wav.npy")

In [5]:
metadata_df = pd.read_csv("../UrbanSound8K.csv")
metadata_df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [17]:
def get_fold_df(file_limit=None):
    fold_df = pd.DataFrame(columns=["fold", "filename", "classID"])
    for i in range(1, 11):
        dirname = os.path.join(root_dir, f'fold{i}')
        filenames = [f for f in os.listdir(dirname) if os.path.isfile(os.path.join(dirname, f))]
        for j, f in enumerate(filenames):
            if file_limit is not None and j >= file_limit:
                break
            audio_filename = f[:-4]
            class_id = metadata_df.loc[metadata_df['slice_file_name'] == audio_filename, "classID"].values[0]
            fold_df = fold_df.append({"fold": i, "filename": f, "classID": class_id}, ignore_index=True)
    return fold_df


In [18]:
fold_df = get_fold_df()

In [19]:
fold_df.head()

,fold,filename,classID
0,1,24074-1-0-11.wav.npy,1
1,1,176787-5-0-6.wav.npy,5
2,1,78360-4-0-0.wav.npy,4
3,1,86279-3-0-0.wav.npy,3
4,1,55020-4-0-0.wav.npy,4


In [22]:
class SignalDataset(Dataset):
    def __init__(self, fold_df):
        self.fold_df = fold_df

    def __len__(self):
        return len(self.fold_df)

    def __getitem__(self, idx):
        row = self.fold_df[idx]
        filename = row['filename']